In [1]:
from sentence_transformers import SentenceTransformer
from PIL import Image

# Load CLIP model
model = SentenceTransformer("clip-ViT-B-32")

/Users/yujian/Documents/workspace/advanced-rag-applications-with-vector-databases-3886256/ch2/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/yujian/Documents/workspace/advanced-rag-applications-with-vector-databases-3886256/ch2/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
import os
from glob import glob


In [3]:
def generate_clip_embeddings(images_path, model, default_size= (128, 128)):

    image_paths = glob(os.path.join(images_path, '**/*.jpeg'), recursive=True)
    
    embeddings = []
    for img_path in image_paths:
        image = Image.open(img_path).resize(default_size)
        embedding = model.encode(image)
        embeddings.append(embedding)

    return embeddings, image_paths

In [4]:
embeddings, image_paths = generate_clip_embeddings(images_path="../images", model=model)

In [5]:
import faiss
import numpy as np

In [9]:
dimension = len(embeddings[0])
index = faiss.IndexFlatIP(dimension)
index = faiss.IndexIDMap(index)

vectors = np.array(embeddings).astype(np.float32)

# Add vectors to the index with IDs
index.add_with_ids(vectors, np.array(range(len(embeddings))))

# Save the index
faiss.write_index(index, output_path)
print(f"Index created and saved to {output_path}")

# Save image paths
with open(output_path + '.paths', 'w') as f:
    for img_path in image_paths:
        f.write(img_path + '\n')

Index created and saved to ./indices/vector.index
